In [1]:
import pandas as pd

# Load the CSV file with different encoding to handle potential issues
file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data.csv'
try:
    df = pd.read_csv(file_path, encoding='latin1')
except Exception as e:
    print(f"Error reading the file: {e}")

# Display the first few rows of the dataframe to understand its structure
try:
    print("Column names:", df.columns.tolist())
    print(df.head())
except Exception as e:
    print(f"Error displaying the dataframe: {e}")

Column names: ['match id', 'Player Id', 'match_dt', 'team1', 'team2', 'winner', 'by', 'win amount', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season', 'ground_id', 'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls', 'team1_id', 'team2_id', 'winner_id', 'inning_x', 'runs_x', 'balls_faced', 'Fours_x', 'Sixes_x']
   match id  Player Id    match_dt  team1 team2 winner       by  win amount  \
0   8638034    7907451  01/01/2021  Nn Ds    Wn     Wn  wickets           9   
1   8638034    4381761  01/01/2021  Nn Ds    Wn     Wn  wickets           9   
2   8638034      31464  01/01/2021  Nn Ds    Wn     Wn  wickets           9   
3   8638034     258649  01/01/2021  Nn Ds    Wn     Wn  wickets           9   
4   8638034    4949790  01/01/2021  Nn Ds    Wn     Wn  wickets           9   

  toss winner toss decision  ... inning2_wickets inning2_balls team1_id  \
0          Wn         field  

In [7]:
import pandas as pd

# Load the dataset from the provided file with error handling
def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig', on_bad_lines='skip')
        return df
    except UnicodeDecodeError:
        print("Error: Unable to decode file. Please check the file encoding.")
        return None
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
        return None
    except pd.errors.ParserError as e:
        print(f"Error parsing file: {e}")
        return None

# Convert 'match_dt' column to datetime format with error handling
def convert_match_date(df):
    try:
        df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['match_dt'])  # Drop rows where date conversion failed
        df.sort_values(by='match_dt', inplace=True)
    except KeyError:
        print("Error: 'match_dt' column not found in the dataset.")
        return None
    except ValueError as e:
        print(f"Error: Date format incorrect. Details: {e}")
        return None
    return df

# Function to calculate and update rolling averages for a team
def calculate_rolling_average(df, team_id):
    team_data = df[(df['team1_id'] == team_id) | (df['team2_id'] == team_id)].copy()
    team_data = team_data.sort_values(by='match_dt').reset_index(drop=True)
    team_data = team_data.drop_duplicates(subset=['match id'])
    
    rolling_avgs = {}
    previous_scores = []

    for i in range(len(team_data)):
        match_id = team_data.iloc[i]['match id']
        current_match_date = team_data.iloc[i]['match_dt']
        matches_excluding_current = team_data[team_data['match_dt'] < current_match_date]

        # print(f"Processing Match ID {match_id} for Team {team_id} on {current_match_date}")
        # print(f"Previous matches excluding current match date:")
        # print(matches_excluding_current[['match id', 'match_dt', 'inning1_runs', 'inning2_runs']])
        
        if len(matches_excluding_current) > 0:
            scores = []
            for idx, row in matches_excluding_current.iterrows():
                if row['team1_id'] == team_id:
                    scores.append(row['inning1_runs'])
                if row['team2_id'] == team_id:
                    scores.append(row['inning2_runs'])
            
            previous_scores = scores[-5:] if len(scores) > 5 else scores
            rolling_average = pd.Series(previous_scores).mean()
            rolling_avgs[match_id] = rolling_average
            # print(f"Match ID {match_id}: Calculated rolling average for team {team_id}: {rolling_average}. Previous scores: {previous_scores}")
        else:
            rolling_avgs[match_id] = None
            # print(f"Match ID {match_id}: No previous matches for team {team_id}.")
    
    return rolling_avgs

# Function to add rolling average columns to the original dataframe
def add_rolling_average_columns(df):
    # Initialize rolling average columns with NaN
    df['rolling_avg_team1'] = pd.NA
    df['rolling_avg_team2'] = pd.NA
    
    # Process each team separately for team1_id and team2_id
    team1_ids = df['team1_id'].unique()
    team2_ids = df['team2_id'].unique()
    
    for team_id in team1_ids:
        # print(f"Processing team {team_id} for rolling_avg_team1.")
        rolling_avgs_team1 = calculate_rolling_average(df, team_id)
        for match_id, avg in rolling_avgs_team1.items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'rolling_avg_team1'] = avg

    for team_id in team2_ids:
        # print(f"Processing team {team_id} for rolling_avg_team2.")
        rolling_avgs_team2 = calculate_rolling_average(df, team_id)
        for match_id, avg in rolling_avgs_team2.items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'rolling_avg_team2'] = avg

    return df

# Function to update and save the modified dataframe to a new CSV file
def update_and_save_to_csv(df, file_path):
    if df is not None:
        df = add_rolling_average_columns(df)
        if df is not None:
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"Updated dataframe with rolling averages saved to {file_path}")
        else:
            print("Error: Could not calculate rolling average columns.")
    else:
        print("Error: DataFrame is None. Cannot proceed with updating and saving.")

# Main function to execute the workflow
def main():
    # Load the dataset
    file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data.csv'
    df = load_dataset(file_path)
    
    if df is not None:
        # Print the columns to debug
        print("Columns in DataFrame:", df.columns.tolist())

        # Convert match date
        df = convert_match_date(df)
        
        if df is not None:
            # Perform update and save to CSV
            new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_batting Avg.csv'
            update_and_save_to_csv(df, new_file_path)

# Execute the main function
if __name__ == "__main__":
    main()


Columns in DataFrame: ['match id', 'Player Id', 'match_dt', 'team1', 'team2', 'winner', 'by', 'win amount', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season', 'ground_id', 'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls', 'team1_id', 'team2_id', 'winner_id', 'inning_x', 'runs_x', 'balls_faced', 'Fours_x', 'Sixes_x']


/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/689905764.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/689905764.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')


Updated dataframe with rolling averages saved to /Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_batting Avg.csv


# Batting avg and strike rate

In [9]:
import pandas as pd

# Load the dataset from the provided file with error handling
def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig', on_bad_lines='skip')
        return df
    except UnicodeDecodeError:
        print("Error: Unable to decode file. Please check the file encoding.")
        return None
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
        return None
    except pd.errors.ParserError as e:
        print(f"Error parsing file: {e}")
        return None

# Convert 'match_dt' column to datetime format with error handling
def convert_match_date(df):
    try:
        df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['match_dt'])  # Drop rows where date conversion failed
        df.sort_values(by='match_dt', inplace=True)
    except KeyError:
        print("Error: 'match_dt' column not found in the dataset.")
        return None
    except ValueError as e:
        print(f"Error: Date format incorrect. Details: {e}")
        return None
    return df

# Function to calculate rolling averages for runs scored by a team
def calculate_rolling_average(df, team_id):
    team_data = df[(df['team1_id'] == team_id) | (df['team2_id'] == team_id)].copy()
    team_data = team_data.sort_values(by='match_dt').reset_index(drop=True)
    team_data = team_data.drop_duplicates(subset=['match id'])
    
    rolling_avgs = {}
    previous_scores = []

    for i in range(len(team_data)):
        match_id = team_data.iloc[i]['match id']
        current_match_date = team_data.iloc[i]['match_dt']
        matches_excluding_current = team_data[team_data['match_dt'] < current_match_date]

        # Debugging print statements
        # print(f"Processing Match ID {match_id} for Team {team_id} on {current_match_date}")
        # print(f"Previous matches excluding current match date:")
        # print(matches_excluding_current[['match id', 'match_dt', 'inning1_runs', 'inning2_runs']])
        
        if len(matches_excluding_current) > 0:
            scores = []
            for idx, row in matches_excluding_current.iterrows():
                if row['team1_id'] == team_id:
                    scores.append(row['inning1_runs'])
                if row['team2_id'] == team_id:
                    scores.append(row['inning2_runs'])
            
            previous_scores = scores[-5:] if len(scores) > 5 else scores
            rolling_average = pd.Series(previous_scores).mean()
            rolling_avgs[match_id] = rolling_average
            # print(f"Match ID {match_id}: Calculated rolling average for team {team_id}: {rolling_average}. Previous scores: {previous_scores}")
        else:
            rolling_avgs[match_id] = None
            # print(f"Match ID {match_id}: No previous matches for team {team_id}.")
    
    return rolling_avgs

# Function to calculate rolling strike rate for a team
def calculate_rolling_strike_rate(df, team_id):
    team_data = df[(df['team1_id'] == team_id) | (df['team2_id'] == team_id)].copy()
    team_data = team_data.sort_values(by='match_dt').reset_index(drop=True)
    team_data = team_data.drop_duplicates(subset=['match id'])
    
    rolling_strike_rates = {}
    previous_strike_rates = []

    for i in range(len(team_data)):
        match_id = team_data.iloc[i]['match id']
        current_match_date = team_data.iloc[i]['match_dt']
        matches_excluding_current = team_data[team_data['match_dt'] < current_match_date]

        # Debugging print statements
        # print(f"Processing Match ID {match_id} for Team {team_id} on {current_match_date}")
        # print(f"Previous matches excluding current match date:")
        # print(matches_excluding_current[['match id', 'match_dt', 'inning1_runs', 'inning1_balls', 'inning2_runs', 'inning2_balls']])
        
        if len(matches_excluding_current) > 0:
            strike_rates = []
            for idx, row in matches_excluding_current.iterrows():
                if row['team1_id'] == team_id:
                    runs_scored = row['inning1_runs']
                    balls_faced = min(row['inning1_balls'], 120)
                    strike_rate = (runs_scored / balls_faced) * 100 if balls_faced > 0 else 0
                    strike_rates.append(strike_rate)
                if row['team2_id'] == team_id:
                    runs_scored = row['inning2_runs']
                    balls_faced = min(row['inning2_balls'], 120)
                    strike_rate = (runs_scored / balls_faced) * 100 if balls_faced > 0 else 0
                    strike_rates.append(strike_rate)
            
            previous_strike_rates = strike_rates[-5:] if len(strike_rates) > 5 else strike_rates
            rolling_strike_rate = pd.Series(previous_strike_rates).mean()
            rolling_strike_rates[match_id] = rolling_strike_rate
            # print(f"Match ID {match_id}: Calculated rolling strike rate for team {team_id}: {rolling_strike_rate}. Previous strike rates: {previous_strike_rates}")
        else:
            rolling_strike_rates[match_id] = None
            # print(f"Match ID {match_id}: No previous matches for team {team_id}.")
    
    return rolling_strike_rates

# Function to add rolling average columns to the original dataframe
def add_rolling_average_columns(df):
    # Initialize rolling average columns with NaN
    df['rolling_avg_team1'] = pd.NA
    df['rolling_avg_team2'] = pd.NA
    
    # Process each team separately for team1_id and team2_id
    team1_ids = df['team1_id'].unique()
    team2_ids = df['team2_id'].unique()
    
    for team_id in team1_ids:
        # print(f"Processing team {team_id} for rolling_avg_team1.")
        rolling_avgs_team1 = calculate_rolling_average(df, team_id)
        for match_id, avg in rolling_avgs_team1.items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'rolling_avg_team1'] = avg

    for team_id in team2_ids:
        # print(f"Processing team {team_id} for rolling_avg_team2.")
        rolling_avgs_team2 = calculate_rolling_average(df, team_id)
        for match_id, avg in rolling_avgs_team2.items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'rolling_avg_team2'] = avg

    return df

# Function to add rolling strike rate columns to the original dataframe
def add_rolling_strike_rate_columns(df):
    # Initialize rolling strike rate columns with NaN
    df['rolling_strike_rate_team1'] = pd.NA
    df['rolling_strike_rate_team2'] = pd.NA
    
    # Process each team separately for team1_id and team2_id
    team1_ids = df['team1_id'].unique()
    team2_ids = df['team2_id'].unique()
    
    for team_id in team1_ids:
        # print(f"Processing team {team_id} for rolling_strike_rate_team1.")
        rolling_strike_rates_team1 = calculate_rolling_strike_rate(df, team_id)
        for match_id, avg in rolling_strike_rates_team1.items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'rolling_strike_rate_team1'] = avg

    for team_id in team2_ids:
        # print(f"Processing team {team_id} for rolling_strike_rate_team2.")
        rolling_strike_rates_team2 = calculate_rolling_strike_rate(df, team_id)
        for match_id, avg in rolling_strike_rates_team2.items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'rolling_strike_rate_team2'] = avg

    return df

# Function to update and save the modified dataframe to a new CSV file
def update_and_save_to_csv(df, file_path):
    if df is not None:
        df = add_rolling_average_columns(df)
        df = add_rolling_strike_rate_columns(df)
        if df is not None:
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"Updated dataframe with rolling averages and strike rates saved to {file_path}")
        else:
            print("Error: Could not calculate rolling average columns.")
    else:
        print("Error: DataFrame is None. Cannot proceed with updating and saving.")

# Main function to execute the workflow
def main():
    # Load the dataset
    file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data.csv'
    df = load_dataset(file_path)
    
    if df is not None:
        # Print the columns to debug
        print("Columns in DataFrame:", df.columns.tolist())

        # Convert match date
        df = convert_match_date(df)
        
        if df is not None:
            # Perform update and save to CSV
            new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Batting Done.csv'
            update_and_save_to_csv(df, new_file_path)

# Execute the main function
if __name__ == "__main__":
    main()


Columns in DataFrame: ['match id', 'Player Id', 'match_dt', 'team1', 'team2', 'winner', 'by', 'win amount', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season', 'ground_id', 'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls', 'team1_id', 'team2_id', 'winner_id', 'inning_x', 'runs_x', 'balls_faced', 'Fours_x', 'Sixes_x']


/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/995929707.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/995929707.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')


Updated dataframe with rolling averages and strike rates saved to /Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Batting Done.csv


# Boundary Instances

In [17]:
import pandas as pd

# Load the dataset from the provided file with error handling
def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig', on_bad_lines='skip')
        return df
    except UnicodeDecodeError:
        print("Error: Unable to decode file. Please check the file encoding.")
        return None
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
        return None
    except pd.errors.ParserError as e:
        print(f"Error parsing file: {e}")
        return None

# Convert 'match_dt' column to datetime format with error handling
def convert_match_date(df):
    try:
        df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['match_dt'])  # Drop rows where date conversion failed
        df.sort_values(by='match_dt', inplace=True)
    except KeyError:
        print("Error: 'match_dt' column not found in the dataset.")
        return None
    return df

# Function to calculate the number of 30+, 50+, and 100+ scores for a team in the last 5 matches
def calculate_score_instances(df, team_id):
    # Filter the team data considering both innings
    team_data = df[((df['team1_id'] == team_id) & (df['inning_x'] == 1)) | 
                   ((df['team2_id'] == team_id) & (df['inning_x'] == 2))].copy()
    team_data = team_data.sort_values(by='match_dt').reset_index(drop=True)

    score_instances = {
        '30+': {},
        '50+': {},
        '100+': {}
    }

    for i in range(len(team_data)):
        match_id = team_data.iloc[i]['match id']
        current_match_date = team_data.iloc[i]['match_dt']
        matches_excluding_current = team_data[team_data['match_dt'] < current_match_date]

        # Select only the last 5 distinct matches
        last_five_matches_ids = matches_excluding_current.drop_duplicates(subset=['match id'], keep='last').tail(5)['match id']
        last_five_matches = matches_excluding_current[matches_excluding_current['match id'].isin(last_five_matches_ids)]

        if len(last_five_matches) > 0:
            scores_30 = last_five_matches[last_five_matches['runs_x'] >= 30].shape[0]
            scores_50 = last_five_matches[last_five_matches['runs_x'] >= 50].shape[0]
            scores_100 = last_five_matches[last_five_matches['runs_x'] >= 100].shape[0]
            
            score_instances['30+'][match_id] = scores_30
            score_instances['50+'][match_id] = scores_50
            score_instances['100+'][match_id] = scores_100
        else:
            score_instances['30+'][match_id] = 0
            score_instances['50+'][match_id] = 0
            score_instances['100+'][match_id] = 0
    
    return score_instances

# Function to add columns for 30+, 50+, and 100+ scores to the original dataframe
def add_score_instance_columns(df):
    # Initialize columns with NaN
    df['no_30+score_team_last5_team1'] = pd.NA
    df['no_50+score_team_last5_team1'] = pd.NA
    df['no_100+score_team_last5_team1'] = pd.NA
    df['no_30+score_team_last5_team2'] = pd.NA
    df['no_50+score_team_last5_team2'] = pd.NA
    df['no_100+score_team_last5_team2'] = pd.NA
    
    # Process each team separately for team1_id and team2_id
    for team_id in df['team1_id'].unique():
        score_instances_team1 = calculate_score_instances(df, team_id)
        
        for match_id, count in score_instances_team1['30+'].items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'no_30+score_team_last5_team1'] = count
        for match_id, count in score_instances_team1['50+'].items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'no_50+score_team_last5_team1'] = count
        for match_id, count in score_instances_team1['100+'].items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'no_100+score_team_last5_team1'] = count

    for team_id in df['team2_id'].unique():
        score_instances_team2 = calculate_score_instances(df, team_id)
        
        for match_id, count in score_instances_team2['30+'].items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'no_30+score_team_last5_team2'] = count
        for match_id, count in score_instances_team2['50+'].items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'no_50+score_team_last5_team2'] = count
        for match_id, count in score_instances_team2['100+'].items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'no_100+score_team_last5_team2'] = count

    return df

# Main function to execute the workflow
def main():
    # Load the dataset
    file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data.csv'
    df = load_dataset(file_path)
    
    if df is not None:
        # Print the columns to debug
        print("Columns in DataFrame:", df.columns.tolist())

        # Convert match date
        df = convert_match_date(df)
        
        if df is not None:
            # Perform update and save to CSV
            new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Batting Instances.csv'
            df = add_score_instance_columns(df)
            df.to_csv(new_file_path, index=False, encoding='utf-8-sig')
            print(f"Updated dataframe with score instances saved to {new_file_path}")

# Execute the main function
if __name__ == "__main__":
    main()


Columns in DataFrame: ['match id', 'Player Id', 'match_dt', 'team1', 'team2', 'winner', 'by', 'win amount', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season', 'ground_id', 'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls', 'team1_id', 'team2_id', 'winner_id', 'inning_x', 'runs_x', 'balls_faced', 'Fours_x', 'Sixes_x']


/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/1980040472.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/1980040472.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')


Updated dataframe with score instances saved to /Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Batting Instances.csv


# Boundary Count

In [19]:
import pandas as pd

# Load the dataset from the provided file with error handling
def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig', on_bad_lines='skip')
        return df
    except UnicodeDecodeError:
        print("Error: Unable to decode file. Please check the file encoding.")
        return None
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
        return None
    except pd.errors.ParserError as e:
        print(f"Error parsing file: {e}")
        return None

# Convert 'match_dt' column to datetime format with error handling
def convert_match_date(df):
    try:
        df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['match_dt'])  # Drop rows where date conversion failed
        df.sort_values(by='match_dt', inplace=True)
    except KeyError:
        print("Error: 'match_dt' column not found in the dataset.")
        return None
    return df

# Function to calculate the number of 4s and 6s hit by batsmen in the last 5 matches
def calculate_boundaries_instances(df, team_id):
    # Filter the team data considering both innings
    team_data = df[((df['team1_id'] == team_id) & (df['inning_x'] == 1)) | 
                   ((df['team2_id'] == team_id) & (df['inning_x'] == 2))].copy()
    team_data = team_data.sort_values(by='match_dt').reset_index(drop=True)

    boundaries_instances = {
        '4s': {},
        '6s': {}
    }

    for i in range(len(team_data)):
        match_id = team_data.iloc[i]['match id']
        current_match_date = team_data.iloc[i]['match_dt']
        matches_excluding_current = team_data[team_data['match_dt'] < current_match_date]

        # Select only the last 5 distinct matches
        last_five_matches_ids = matches_excluding_current.drop_duplicates(subset=['match id'], keep='last').tail(5)['match id']
        last_five_matches = matches_excluding_current[matches_excluding_current['match id'].isin(last_five_matches_ids)]

        if len(last_five_matches) > 0:
            fours_hit = last_five_matches['Fours_x'].sum()
            sixes_hit = last_five_matches['Sixes_x'].sum()
            
            boundaries_instances['4s'][match_id] = fours_hit
            boundaries_instances['6s'][match_id] = sixes_hit
        else:
            boundaries_instances['4s'][match_id] = 0
            boundaries_instances['6s'][match_id] = 0
    
    return boundaries_instances

# Function to add columns for 4s and 6s to the original dataframe
def add_boundaries_instance_columns(df):
    # Initialize columns with NaN
    df['boundaries_hit_team1'] = pd.NA
    df['boundaries_hit_team2'] = pd.NA
    df['sixes_hit_team1'] = pd.NA
    df['sixes_hit_team2'] = pd.NA
    
    # Process each team separately for team1_id and team2_id
    for team_id in df['team1_id'].unique():
        boundaries_instances_team1 = calculate_boundaries_instances(df, team_id)
        
        for match_id, count in boundaries_instances_team1['4s'].items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'boundaries_hit_team1'] = count
        for match_id, count in boundaries_instances_team1['6s'].items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'sixes_hit_team1'] = count

    for team_id in df['team2_id'].unique():
        boundaries_instances_team2 = calculate_boundaries_instances(df, team_id)
        
        for match_id, count in boundaries_instances_team2['4s'].items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'boundaries_hit_team2'] = count
        for match_id, count in boundaries_instances_team2['6s'].items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'sixes_hit_team2'] = count

    return df

# Main function to execute the workflow
def main():
    # Load the dataset
    file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data.csv'
    df = load_dataset(file_path)
    
    if df is not None:
        # Print the columns to debug
        print("Columns in DataFrame:", df.columns.tolist())

        # Convert match date
        df = convert_match_date(df)
        
        if df is not None:
            # Perform update and save to CSV
            new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Boundary Instance.csv'
            df = add_boundaries_instance_columns(df)
            df.to_csv(new_file_path, index=False, encoding='utf-8-sig')
            print(f"Updated dataframe with boundaries instances saved to {new_file_path}")

# Execute the main function
if __name__ == "__main__":
    main()


Columns in DataFrame: ['match id', 'Player Id', 'match_dt', 'team1', 'team2', 'winner', 'by', 'win amount', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season', 'ground_id', 'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls', 'team1_id', 'team2_id', 'winner_id', 'inning_x', 'runs_x', 'balls_faced', 'Fours_x', 'Sixes_x']


/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/531500050.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/531500050.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')


Updated dataframe with boundaries instances saved to /Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Boundary Instance.csv


# Rolling bowler strike rate last 5

In [21]:
import pandas as pd

# Load the dataset from the provided file with error handling
def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig', on_bad_lines='skip')
        return df
    except UnicodeDecodeError:
        print("Error: Unable to decode file. Please check the file encoding.")
        return None
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
        return None
    except pd.errors.ParserError as e:
        print(f"Error parsing file: {e}")
        return None

# Convert 'match_dt' column to datetime format with error handling
def convert_match_date(df):
    try:
        df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['match_dt'])  # Drop rows where date conversion failed
        df.sort_values(by='match_dt', inplace=True)
    except KeyError:
        print("Error: 'match_dt' column not found in the dataset.")
        return None
    return df

# Function to calculate the rolling average of bowling strike rate for a team
def calculate_rolling_bowl_strike_rate(df, team_id):
    # Filter the team data considering both innings
    team_data = df[(df['team1_id'] == team_id) | (df['team2_id'] == team_id)].copy()
    team_data = team_data.sort_values(by='match_dt').reset_index(drop=True)

    bowl_strike_rate = {}
    
    for i in range(len(team_data)):
        match_id = team_data.iloc[i]['match id']
        current_match_date = team_data.iloc[i]['match_dt']
        matches_excluding_current = team_data[team_data['match_dt'] < current_match_date]

        # Drop duplicates based on match_id
        matches_excluding_current = matches_excluding_current.drop_duplicates(subset=['match id'])

        # Select only the last 5 distinct matches
        last_five_matches_ids = matches_excluding_current.tail(5)['match id']
        last_five_matches = matches_excluding_current[matches_excluding_current['match id'].isin(last_five_matches_ids)]

        if len(last_five_matches) > 0:
            total_balls_faced = last_five_matches['inning2_balls'].clip(upper=120).sum()
            total_wickets = last_five_matches['inning2_wickets'].sum()
            bowl_strike_rate_value = total_balls_faced / total_wickets if total_wickets > 0 else 0
            
            bowl_strike_rate[match_id] = bowl_strike_rate_value
        else:
            bowl_strike_rate[match_id] = 0
    
    return bowl_strike_rate

# Function to add columns for rolling average of bowling strike rate to the original dataframe
def add_rolling_bowl_strike_rate_columns(df):
    # Initialize columns with NaN
    df['rolling_avg_bowl_strRate_team1'] = pd.NA
    df['rolling_avg_bowl_strRate_team2'] = pd.NA
    
    # Process each team separately for team1_id and team2_id
    team_ids = pd.concat([df['team1_id'], df['team2_id']]).unique()
    
    for team_id in team_ids:
        bowl_strike_rate = calculate_rolling_bowl_strike_rate(df, team_id)
        
        for match_id, rate in bowl_strike_rate.items():
            if df.loc[df['match id'] == match_id, 'team1_id'].values[0] == team_id:
                df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'rolling_avg_bowl_strRate_team1'] = rate
            elif df.loc[df['match id'] == match_id, 'team2_id'].values[0] == team_id:
                df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'rolling_avg_bowl_strRate_team2'] = rate

    return df

# Main function to execute the workflow
def main():
    # Load the dataset
    file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data.csv'
    df = load_dataset(file_path)
    
    if df is not None:
        # Print the columns to debug
        print("Columns in DataFrame:", df.columns.tolist())

        # Convert match date
        df = convert_match_date(df)
        
        if df is not None:
            # Perform update and save to CSV
            new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Bowling_Strike_Rate.csv'
            df = add_rolling_bowl_strike_rate_columns(df)
            df.to_csv(new_file_path, index=False, encoding='utf-8-sig')
            print(f"Updated dataframe with rolling average bowling strike rate saved to {new_file_path}")

# Execute the main function
if __name__ == "__main__":
    main()


Columns in DataFrame: ['match id', 'Player Id', 'match_dt', 'team1', 'team2', 'winner', 'by', 'win amount', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season', 'ground_id', 'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls', 'team1_id', 'team2_id', 'winner_id', 'inning_x', 'runs_x', 'balls_faced', 'Fours_x', 'Sixes_x']


/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/4122290275.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/4122290275.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')


Updated dataframe with rolling average bowling strike rate saved to /Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Bowling_Strike_Rate.csv


In [22]:
import pandas as pd

# Load the dataset from the provided file with error handling
def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig', on_bad_lines='skip')
        return df
    except UnicodeDecodeError:
        print("Error: Unable to decode file. Please check the file encoding.")
        return None
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
        return None
    except pd.errors.ParserError as e:
        print(f"Error parsing file: {e}")
        return None

# Convert 'match_dt' column to datetime format with error handling
def convert_match_date(df):
    try:
        df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['match_dt'])  # Drop rows where date conversion failed
        df.sort_values(by='match_dt', inplace=True)
    except KeyError:
        print("Error: 'match_dt' column not found in the dataset.")
        return None
    return df

# Function to calculate the rolling average of bowler economy for a team
def calculate_rolling_bowler_economy(df, team_id):
    # Filter the team data considering both innings
    team_data = df[(df['team1_id'] == team_id) | (df['team2_id'] == team_id)].copy()
    team_data = team_data.sort_values(by='match_dt').reset_index(drop=True)

    bowler_economy = {}
    
    for i in range(len(team_data)):
        match_id = team_data.iloc[i]['match id']
        current_match_date = team_data.iloc[i]['match_dt']
        matches_excluding_current = team_data[team_data['match_dt'] < current_match_date]

        # Drop duplicates based on match_id
        matches_excluding_current = matches_excluding_current.drop_duplicates(subset=['match id'])

        # Select only the last 5 distinct matches
        last_five_matches_ids = matches_excluding_current.tail(5)['match id']
        last_five_matches = matches_excluding_current[matches_excluding_current['match id'].isin(last_five_matches_ids)]

        if len(last_five_matches) > 0:
            total_runs_conceded = last_five_matches['inning2_runs'].sum()
            total_balls_faced = last_five_matches['inning2_balls'].clip(upper=120).sum()
            bowler_economy_value = (total_runs_conceded / total_balls_faced) * 6 if total_balls_faced > 0 else 0
            
            bowler_economy[match_id] = bowler_economy_value
        else:
            bowler_economy[match_id] = 0
    
    return bowler_economy

# Function to add columns for rolling average of bowler economy to the original dataframe
def add_rolling_bowler_economy_columns(df):
    # Initialize columns with NaN
    df['avg_bowler_economy_last5_team1'] = pd.NA
    df['avg_bowler_economy_last5_team2'] = pd.NA
    
    # Process each team separately for team1_id and team2_id
    team_ids = pd.concat([df['team1_id'], df['team2_id']]).unique()
    
    for team_id in team_ids:
        bowler_economy = calculate_rolling_bowler_economy(df, team_id)
        
        for match_id, rate in bowler_economy.items():
            if df.loc[df['match id'] == match_id, 'team1_id'].values[0] == team_id:
                df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'avg_bowler_economy_last5_team1'] = rate
            elif df.loc[df['match id'] == match_id, 'team2_id'].values[0] == team_id:
                df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'avg_bowler_economy_last5_team2'] = rate

    return df

# Main function to execute the workflow
def main():
    # Load the dataset
    file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data.csv'
    df = load_dataset(file_path)
    
    if df is not None:
        # Print the columns to debug
        print("Columns in DataFrame:", df.columns.tolist())

        # Convert match date
        df = convert_match_date(df)
        
        if df is not None:
            # Perform update and save to CSV
            new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Bowler_Economy.csv'
            df = add_rolling_bowler_economy_columns(df)
            df.to_csv(new_file_path, index=False, encoding='utf-8-sig')
            print(f"Updated dataframe with rolling average bowler economy saved to {new_file_path}")

# Execute the main function
if __name__ == "__main__":
    main()


Columns in DataFrame: ['match id', 'Player Id', 'match_dt', 'team1', 'team2', 'winner', 'by', 'win amount', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season', 'ground_id', 'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls', 'team1_id', 'team2_id', 'winner_id', 'inning_x', 'runs_x', 'balls_faced', 'Fours_x', 'Sixes_x']


/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/3178739026.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')
/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/3178739026.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['match_dt'] = pd.to_datetime(df['match_dt'], infer_datetime_format=True, errors='coerce')


Updated dataframe with rolling average bowler economy saved to /Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Bowler_Economy.csv


In [23]:
import pandas as pd

# Load the dataset from the provided file with error handling
def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig', on_bad_lines='skip')
        return df
    except UnicodeDecodeError:
        print("Error: Unable to decode file. Please check the file encoding.")
        return None
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
        return None
    except pd.errors.ParserError as e:
        print(f"Error parsing file: {e}")
        return None

# Convert 'match_dt' column to datetime format with error handling
def convert_match_date(df):
    try:
        df['match_dt'] = pd.to_datetime(df['match_dt'], format='%d/%m/%Y')
        df.sort_values(by='match_dt', inplace=True)
    except KeyError:
        print("Error: 'match_dt' column not found in the dataset.")
        return None
    except ValueError:
        print("Error: Date format incorrect. Please ensure the date format is '%d/%m/%Y'.")
        return None
    return df

# Function to calculate the rolling total of wickets taken for a team
def calculate_rolling_wickets_taken(df, team_id):
    team_data = df[(df['team1_id'] == team_id) | (df['team2_id'] == team_id)].copy()
    team_data = team_data.sort_values(by='match_dt').reset_index(drop=True)

    wickets_taken = {}

    for i in range(len(team_data)):
        match_id = team_data.iloc[i]['match id']
        current_match_date = team_data.iloc[i]['match_dt']
        matches_excluding_current = team_data[team_data['match_dt'] < current_match_date]

        matches_excluding_current = matches_excluding_current.drop_duplicates(subset=['match id'])

        last_five_matches_ids = matches_excluding_current.tail(5)['match id']
        last_five_matches = matches_excluding_current[matches_excluding_current['match id'].isin(last_five_matches_ids)]

        total_wickets = 0
        for _, row in last_five_matches.iterrows():
            if row['team1_id'] == team_id:
                total_wickets += row['inning2_wickets']
            if row['team2_id'] == team_id:
                total_wickets += row['inning1_wickets']

        wickets_taken[match_id] = total_wickets

    return wickets_taken

# Function to add columns for rolling total of wickets taken and extras conceded to the original dataframe
def add_rolling_wickets_and_extras_columns(df):
    df['wickets_taken_last5_team1'] = pd.NA
    df['wickets_taken_last5_team2'] = pd.NA

    for team_id in df['team1_id'].unique():
        wickets_taken = calculate_rolling_wickets_taken(df, team_id)

        for match_id, count in wickets_taken.items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'wickets_taken_last5_team1'] = count

    for team_id in df['team2_id'].unique():
        wickets_taken = calculate_rolling_wickets_taken(df, team_id)

        for match_id, count in wickets_taken.items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'wickets_taken_last5_team2'] = count

    return df

# Main function to execute the workflow
def main():
    file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data.csv'
    df = load_dataset(file_path)

    if df is not None:
        print("Columns in DataFrame:", df.columns.tolist())

        df = convert_match_date(df)

        if df is not None:
            new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Wickets_and_Extras.csv'
            df = add_rolling_wickets_and_extras_columns(df)
            df.to_csv(new_file_path, index=False, encoding='utf-8-sig')
            print(f"Updated dataframe with rolling total of wickets taken saved to {new_file_path}")

if __name__ == "__main__":
    main()


Columns in DataFrame: ['match id', 'Player Id', 'match_dt', 'team1', 'team2', 'winner', 'by', 'win amount', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season', 'ground_id', 'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls', 'team1_id', 'team2_id', 'winner_id', 'inning_x', 'runs_x', 'balls_faced', 'Fours_x', 'Sixes_x']
Error: Date format incorrect. Please ensure the date format is '%d/%m/%Y'.


In [24]:
import pandas as pd

# Load the dataset from the provided file with error handling
def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig', on_bad_lines='skip')
        return df
    except UnicodeDecodeError:
        print("Error: Unable to decode file. Please check the file encoding.")
        return None
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
        return None
    except pd.errors.ParserError as e:
        print(f"Error parsing file: {e}")
        return None

# Convert 'match_dt' column to datetime format with error handling
def convert_match_date(df):
    try:
        df['match_dt'] = pd.to_datetime(df['match_dt'], errors='coerce')
        df = df.dropna(subset=['match_dt'])  # Drop rows where date conversion failed
        df.sort_values(by='match_dt', inplace=True)
    except KeyError:
        print("Error: 'match_dt' column not found in the dataset.")
        return None
    return df

# Function to calculate the rolling total of wickets taken for a team
def calculate_rolling_wickets_taken(df, team_id):
    team_data = df[(df['team1_id'] == team_id) | (df['team2_id'] == team_id)].copy()
    team_data = team_data.sort_values(by='match_dt').reset_index(drop=True)

    wickets_taken = {}

    for i in range(len(team_data)):
        match_id = team_data.iloc[i]['match id']
        current_match_date = team_data.iloc[i]['match_dt']
        matches_excluding_current = team_data[team_data['match_dt'] < current_match_date]

        matches_excluding_current = matches_excluding_current.drop_duplicates(subset=['match id'])

        last_five_matches_ids = matches_excluding_current.tail(5)['match id']
        last_five_matches = matches_excluding_current[matches_excluding_current['match id'].isin(last_five_matches_ids)]

        total_wickets = 0
        for _, row in last_five_matches.iterrows():
            if row['team1_id'] == team_id:
                total_wickets += row['inning2_wickets']
            if row['team2_id'] == team_id:
                total_wickets += row['inning1_wickets']

        wickets_taken[match_id] = total_wickets

    return wickets_taken

# Function to add columns for rolling total of wickets taken and extras conceded to the original dataframe
def add_rolling_wickets_and_extras_columns(df):
    df['wickets_taken_last5_team1'] = pd.NA
    df['wickets_taken_last5_team2'] = pd.NA

    for team_id in df['team1_id'].unique():
        wickets_taken = calculate_rolling_wickets_taken(df, team_id)

        for match_id, count in wickets_taken.items():
            df.loc[(df['match id'] == match_id) & (df['team1_id'] == team_id), 'wickets_taken_last5_team1'] = count

    for team_id in df['team2_id'].unique():
        wickets_taken = calculate_rolling_wickets_taken(df, team_id)

        for match_id, count in wickets_taken.items():
            df.loc[(df['match id'] == match_id) & (df['team2_id'] == team_id), 'wickets_taken_last5_team2'] = count

    return df

# Main function to execute the workflow
def main():
    file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data.csv'
    df = load_dataset(file_path)

    if df is not None:
        print("Columns in DataFrame:", df.columns.tolist())

        df = convert_match_date(df)

        if df is not None:
            new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Wickets_and_Extras.csv'
            df = add_rolling_wickets_and_extras_columns(df)
            df.to_csv(new_file_path, index=False, encoding='utf-8-sig')
            print(f"Updated dataframe with rolling total of wickets taken saved to {new_file_path}")

if __name__ == "__main__":
    main()


Columns in DataFrame: ['match id', 'Player Id', 'match_dt', 'team1', 'team2', 'winner', 'by', 'win amount', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season', 'ground_id', 'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls', 'team1_id', 'team2_id', 'winner_id', 'inning_x', 'runs_x', 'balls_faced', 'Fours_x', 'Sixes_x']


/var/folders/p7/vv78xffs7mz93tb1wsgmf_7m0000gn/T/ipykernel_53579/190642324.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['match_dt'] = pd.to_datetime(df['match_dt'], errors='coerce')


Updated dataframe with rolling total of wickets taken saved to /Users/pratikrohila/Desktop/AMEX Round 2/Round 2 Final Data_Wickets_and_Extras.csv


In [33]:
import pandas as pd

# Define the file paths
file1_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Test Data Sample/Test Data Sample.csv'
file2_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Features Only Cleaned.xlsx'
updated_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Test Data Sample/updated_data.csv'

# Load the existing CSV file
df_existing = pd.read_csv(file1_path)

# Load the second Excel file
df_new = pd.read_excel(file2_path)

# Define the columns to extract
columns_to_extract = [
    'rolling_avg_team1', 'rolling_avg_team2', 'rolling_strike_rate_team1', 'rolling_strike_rate_team2',
    'no_30+score_team_last5_team1', 'no_50+score_team_last5_team1', 'no_100+score_team_last5_team1',
    'no_30+score_team_last5_team2', 'no_50+score_team_last5_team2', 'no_100+score_team_last5_team2',
    'boundaries_hit_team1', 'boundaries_hit_team2', 'sixes_hit_team1', 'sixes_hit_team2',
    'avg_bowler_economy_last5_team1', 'avg_bowler_economy_last5_team2', 'rolling_avg_bowl_strRate_team1',
    'rolling_avg_bowl_strRate_team2', 'wickets_taken_last5_team1', 'wickets_taken_last5_team2'
]

# Convert match_dt to datetime in both DataFrames
df_existing['match_dt'] = pd.to_datetime(df_existing['match_dt'])
df_new['match_dt'] = pd.to_datetime(df_new['match_dt'])

# Function to extract data for a given team and match date
def extract_team_data(match_dt, team_id, df_new, columns_to_extract):
    print(f"Extracting data for team_id: {team_id} and match_dt: {match_dt}")
    
    # Filter the second DataFrame for the specific team
    df_team = df_new[(df_new['team1_id'] == team_id) | (df_new['team2_id'] == team_id)]
    print(f"Filtered {len(df_team)} matches for team_id: {team_id}")

    # Sort matches before and after the given match_dt
    df_team_before = df_team[df_team['match_dt'] <= match_dt].sort_values(by='match_dt', ascending=False)
    df_team_after = df_team[df_team['match_dt'] > match_dt].sort_values(by='match_dt')

    print(f"Found {len(df_team_before)} matches before and {len(df_team_after)} matches after {match_dt}")

    if not df_team_after.empty:
        first_match_after = df_team_after.iloc[0]
        print(f"First match after {match_dt}: {first_match_after['match_dt']} (team1_id: {first_match_after['team1_id']}, team2_id: {first_match_after['team2_id']})")
        if first_match_after['team1_id'] == team_id:
            print(f"Team {team_id} is playing as team1 in the first match after {match_dt}. Extracting team1 columns.")
            return first_match_after[[col for col in columns_to_extract if 'team1' in col]]
        else:
            print(f"Team {team_id} is playing as team2 in the first match after {match_dt}. Extracting team2 columns.")
            return first_match_after[[col for col in columns_to_extract if 'team2' in col]]
    elif not df_team_before.empty:
        first_match_before = df_team_before.iloc[0]
        print(f"No match after {match_dt}. Using first match before {match_dt}: {first_match_before['match_dt']} (team1_id: {first_match_before['team1_id']}, team2_id: {first_match_before['team2_id']})")
        if first_match_before['team1_id'] == team_id:
            print(f"Team {team_id} is playing as team1 in the first match before {match_dt}. Extracting team1 columns.")
            return first_match_before[[col for col in columns_to_extract if 'team1' in col]]
        else:
            print(f"Team {team_id} is playing as team2 in the first match before {match_dt}. Extracting team2 columns.")
            return first_match_before[[col for col in columns_to_extract if 'team2' in col]]
    print(f"No match found for team_id: {team_id} either before or after {match_dt}")
    return pd.Series([None] * len(columns_to_extract), index=columns_to_extract)

# Iterate over each row in the existing DataFrame
for idx, row in df_existing.iterrows():
    match_dt = row['match_dt']
    
    # Process team1
    team1_id = row['team1_id']
    team1_data = extract_team_data(match_dt, team1_id, df_new, columns_to_extract)
    print(f"Extracted data for team1_id {team1_id}: {team1_data.values}")
    
    if row['team1_id'] == team1_id:
        print(f"Adding extracted data to team1 columns in first file for team1_id {team1_id}.")
        for col in team1_data.index:
            new_col_name = col.replace('team1', 'team1_').replace('team2', 'team1_')
            df_existing.loc[idx, new_col_name] = team1_data[col]
    else:
        print(f"Adding extracted data to team2 columns in first file for team1_id {team1_id}.")
        for col in team1_data.index:
            new_col_name = col.replace('team1', 'team2_').replace('team2', 'team2_')
            df_existing.loc[idx, new_col_name] = team1_data[col]
    
    # Process team2
    team2_id = row['team2_id']
    team2_data = extract_team_data(match_dt, team2_id, df_new, columns_to_extract)
    print(f"Extracted data for team2_id {team2_id}: {team2_data.values}")
    
    if row['team2_id'] == team2_id:
        print(f"Adding extracted data to team2 columns in first file for team2_id {team2_id}.")
        for col in team2_data.index:
            new_col_name = col.replace('team1', 'team2_').replace('team2', 'team2_')
            df_existing.loc[idx, new_col_name] = team2_data[col]
    else:
        print(f"Adding extracted data to team1 columns in first file for team2_id {team2_id}.")
        for col in team2_data.index:
            new_col_name = col.replace('team1', 'team1_').replace('team2', 'team1_')
            df_existing.loc[idx, new_col_name] = team2_data[col]

# Save the updated DataFrame to a new CSV file
df_existing.to_csv(updated_file_path, index=False)

# Display the first few rows of the merged DataFrame
print(df_existing.head())


Extracting data for team_id: 33949 and match_dt: 2023-01-09 00:00:00
Filtered 38 matches for team_id: 33949
Found 24 matches before and 14 matches after 2023-01-09 00:00:00
First match after 2023-01-09 00:00:00: 2023-01-14 00:00:00 (team1_id: 33942, team2_id: 33949)
Team 33949 is playing as team2 in the first match after 2023-01-09 00:00:00. Extracting team2 columns.
Extracted data for team1_id 33949: [144.0 120.0 5 3 1 45 23 6.964467005076143 15.972972972972974 35]
Adding extracted data to team1 columns in first file for team1_id 33949.
Extracting data for team_id: 33928 and match_dt: 2023-01-09 00:00:00
Filtered 40 matches for team_id: 33928
Found 25 matches before and 15 matches after 2023-01-09 00:00:00
First match after 2023-01-09 00:00:00: 2023-01-15 00:00:00 (team1_id: 33963, team2_id: 33928)
Team 33928 is playing as team2 in the first match after 2023-01-09 00:00:00. Extracting team2 columns.
Extracted data for team2_id 33928: [151.8 140.69509403582325 7 3 0 53 28 8.31365313653

# ML Model

In [83]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import joblib

# Load the data
file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Features Only Cleaned.xlsx'
data = pd.read_excel(file_path)

# Ensure match_id is not the index
data.reset_index(drop=True, inplace=True)

# Convert match_date to datetime and extract date features
data['match_date'] = pd.to_datetime(data['match_date'])
data['year'] = data['match_date'].dt.year
data['month'] = data['match_date'].dt.month
data['day_of_week'] = data['match_date'].dt.dayofweek

# Encode categorical features
label_encoder = LabelEncoder()
categorical_columns = ['toss_winner_team_name', 'toss_decision_by_toss_winning_team']

for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column].astype(str))

# Create binary target variable: 1 if team1 wins, 0 if team2 wins
data['target'] = (data['winner_id'] == data['team1_id']).astype(int)

# Prepare the data
features = ['boundaries_hit_team1', 'boundaries_hit_team2',
            'rolling_avg_bowl_strRate_team1', 'rolling_avg_bowl_strRate_team2', 'rolling_avg_team1', 
            'rolling_avg_team2', 'rolling_strike_rate_team1', 'rolling_strike_rate_team2', 
            'toss_winner_team_name', 
            'toss_decision_by_toss_winning_team', 'team1_id', 'team2_id', 'wickets_taken_last5_team2', 'wickets_taken_last5_team1']

X = data[features]
y = data['target']

# Identify and handle missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Sort the data by match_date to ensure correct ordering
data = data.sort_values(by='match_date')

# Define the percentage for training data
train_size = 0.8  # 80% for training, 20% for testing

# Determine the cutoff index for the training set
cutoff_index = int(len(data) * train_size)

# Split the data into training and testing sets based on the cutoff index
train_data = data.iloc[:cutoff_index]
test_data = data.iloc[cutoff_index:]

X_train = train_data[features]
y_train = train_data['target']
X_test = test_data[features]
y_test = test_data['target']

# Impute missing values in train and test sets
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Define the model and hyperparameter grid
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
param_grid = {
    'n_estimators': [200, 400],
    'learning_rate': [0.01, 0.3, 0.5],
    'max_depth': [11, 13, 15],
    'colsample_bytree': [0.2, 0.8]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

# Train the model with the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Save the trained model
joblib.dump(best_model, '/Users/pratikrohila/Desktop/AMEX DATA/xgboost_cricket_model_tuned.pkl')

# Save the label encoder for future use
joblib.dump(label_encoder, '/Users/pratikrohila/Desktop/AMEX DATA/label_encoder.pkl')

Best parameters found:  {'colsample_bytree': 0.2, 'learning_rate': 0.01, 'max_depth': 15, 'n_estimators': 400}
Accuracy: 0.54


['/Users/pratikrohila/Desktop/AMEX DATA/label_encoder.pkl']

In [73]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import joblib

# Load the trained model and label encoder
model = joblib.load('/Users/pratikrohila/Desktop/AMEX DATA/xgboost_cricket_model_tuned.pkl')
label_encoder = joblib.load('/Users/pratikrohila/Desktop/AMEX DATA/label_encoder.pkl')

# Define a function to prepare new match data
def prepare_new_match_data(new_data, label_encoder, features):
    # Encode categorical features
    categorical_columns = ['toss_winner_team_name', 'toss_decision_by_toss_winning_team']
    for column in categorical_columns:
        new_data[column] = new_data[column].astype(str).map(lambda s: s if s in label_encoder.classes_ else '<unknown>')
        new_data[column] = new_data[column].map(lambda s: label_encoder.transform([s])[0] if s in label_encoder.classes_ else -1)
    
    # Identify and handle missing values
    imputer = SimpleImputer(strategy='mean')
    new_data[features] = imputer.fit_transform(new_data[features])
    
    return new_data[features]

# Load the new match data from the provided CSV file
new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Test Data Sample/updated_data.csv'
new_match_data = pd.read_csv(new_file_path)

# Define the feature columns
features = [
    'boundaries_hit_team1_', 'boundaries_hit_team2_',
    'rolling_avg_bowl_strRate_team1_', 'rolling_avg_bowl_strRate_team2_', 
    'rolling_avg_team1_', 'rolling_avg_team2_', 
    'rolling_strike_rate_team1_', 'rolling_strike_rate_team2_', 
    'toss_winner_team_name', 'toss_decision_by_toss_winning_team', 
    'team1_id', 'team2_id', 'wickets_taken_last5_team2_', 
    'wickets_taken_last5_team1_'
]

# Prepare the new match data
X_new = prepare_new_match_data(new_match_data, label_encoder, features)

# Make predictions
predictions = model.predict(X_new)

# Map predictions back to team1_id or team2_id
new_match_data['predicted_winner'] = new_match_data.apply(
    lambda row: row['team1_id'] if predictions[row.name] == 1 else row['team2_id'], axis=1)

# Save the predictions to a new CSV file
output_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Test Data Sample/predicted_winners_New3.csv'
new_match_data.to_csv(output_path, index=False)

# Display the predictions
print(new_match_data[['match_id', 'team1_id', 'team2_id', 'predicted_winner']])


     match_id  team1_id  team2_id  predicted_winner
0     9272619   33949.0   33928.0           33928.0
1     9086958     209.0      69.0              69.0
2     9433654    7573.0    9701.0            7573.0
3     9097248   22784.0   22763.0           22763.0
4     9097234   23841.0   23750.0           23750.0
..        ...       ...       ...               ...
202   9984162   30407.0   36014.0           36014.0
203   9984176   36014.0   30400.0           36014.0
204   9984183   30393.0   30428.0           30428.0
205   9984190   36014.0   30428.0           30428.0
206   9984197   36014.0   30400.0           36014.0

[207 rows x 4 columns]


In [77]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import joblib

# Load the trained model and label encoder
model = joblib.load('/Users/pratikrohila/Desktop/AMEX DATA/xgboost_cricket_model_tuned.pkl')
label_encoder = joblib.load('/Users/pratikrohila/Desktop/AMEX DATA/label_encoder.pkl')

# Define a function to prepare new match data
def prepare_new_match_data(new_data, label_encoder, features):
    # Encode categorical features
    categorical_columns = ['toss_winner_team_name', 'toss_decision_by_toss_winning_team']
    for column in categorical_columns:
        new_data[column] = new_data[column].astype(str).map(lambda s: s if s in label_encoder.classes_ else '<unknown>')
        new_data[column] = new_data[column].map(lambda s: label_encoder.transform([s])[0] if s in label_encoder.classes_ else -1)
    
    # Identify and handle missing values
    imputer = SimpleImputer(strategy='mean')
    new_data[features] = imputer.fit_transform(new_data[features])
    
    return new_data[features]

# Load the new match data from the provided CSV file
new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Test Data Sample/updated_data.csv'
new_match_data = pd.read_csv(new_file_path)

# Define the feature columns expected by the model
features = [
    'boundaries_hit_team1_', 'boundaries_hit_team2_',
    'rolling_avg_bowl_strRate_team1_', 'rolling_avg_bowl_strRate_team2_', 
    'rolling_avg_team1_', 'rolling_avg_team2_', 
    'rolling_strike_rate_team1_', 'rolling_strike_rate_team2_', 
    'no_30+score_team_last5_team1_', 'no_50+score_team_last5_team1_', 
    'no_30+score_team_last5_team2_', 'no_50+score_team_last5_team2_', 
    'toss_winner_team_name', 'toss_decision_by_toss_winning_team'
]

# Add dataset_type with default value
new_match_data['dataset_type'] = 'r1'  # or 'train' based on your context

# Ensure we select only the features expected by the model
new_match_data = new_match_data[features + ['team1_id', 'team2_id', 'match_id', 'dataset_type']]

# Prepare the new match data
X_new = prepare_new_match_data(new_match_data, label_encoder, features)

# Make predictions and get probabilities
predictions = model.predict(X_new)
prediction_probs = model.predict_proba(X_new)[:, 1]  # Probability of class 1

# Map predictions back to team1_id or team2_id
new_match_data['win_pred_team_id'] = new_match_data.apply(
    lambda row: row['team1_id'] if predictions[row.name] == 1 else row['team2_id'], axis=1)

# Add win_pred_score (probability)
new_match_data['win_pred_score'] = prediction_probs

# Add fixed variables
new_match_data['train_algorithm'] = 'xgboost'
new_match_data['Ensemble?'] = 'no'
new_match_data['train_hps_trees'] = 100
new_match_data['train_hps_depth'] = 8
new_match_data['train_hps_lr'] = 0.1

# Add empty columns for dynamic variables
for i in range(1, 11):
    new_match_data[f'indep_feat_id{i}'] = ''

# Create the submission dataframe with required columns
submission = new_match_data[['match_id', 'dataset_type', 'win_pred_team_id', 'win_pred_score', 
                             'train_algorithm', 'Ensemble?', 'train_hps_trees', 'train_hps_depth', 
                             'train_hps_lr'] + [f'indep_feat_id{i}' for i in range(1, 11)]]

# Save the submission dataframe to a new CSV file
output_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Test Data Sample/predicted_winners_New4.csv'
submission.to_csv(output_path, index=False)

# Display the submission dataframe
print(submission.head())


   match_id dataset_type  win_pred_team_id  win_pred_score train_algorithm  \
0   9272619           r1             33949        0.533543         xgboost   
1   9086958           r1                69        0.376358         xgboost   
2   9433654           r1              7573        0.716891         xgboost   
3   9097248           r1             22784        0.559862         xgboost   
4   9097234           r1             23750        0.474241         xgboost   

  Ensemble?  train_hps_trees  train_hps_depth  train_hps_lr indep_feat_id1  \
0        no              100                8           0.1                  
1        no              100                8           0.1                  
2        no              100                8           0.1                  
3        no              100                8           0.1                  
4        no              100                8           0.1                  

  indep_feat_id2 indep_feat_id3 indep_feat_id4 indep_feat_id5 

In [84]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import joblib

# Load the data
file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Features Only Cleaned.xlsx'
data = pd.read_excel(file_path)

# Ensure match_id is not the index
data.reset_index(drop=True, inplace=True)

# Convert match_date to datetime and extract date features
data['match_date'] = pd.to_datetime(data['match_date'])
data['year'] = data['match_date'].dt.year
data['month'] = data['match_date'].dt.month
data['day_of_week'] = data['match_date'].dt.dayofweek

# Encode categorical features
label_encoder = LabelEncoder()
categorical_columns = ['toss_winner_team_name', 'toss_decision_by_toss_winning_team']

for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column].astype(str))

# Create binary target variable: 1 if team1 wins, 0 if team2 wins
data['target'] = (data['winner_id'] == data['team1_id']).astype(int)

# Prepare the data
features = ['boundaries_hit_team1', 'boundaries_hit_team2',
            'rolling_avg_bowl_strRate_team1', 'rolling_avg_bowl_strRate_team2', 'rolling_avg_team1', 
            'rolling_avg_team2', 'rolling_strike_rate_team1', 'rolling_strike_rate_team2', 
            'toss_winner_team_name', 
            'toss_decision_by_toss_winning_team', 'team1_id', 'team2_id', 'wickets_taken_last5_team2', 'wickets_taken_last5_team1']

X = data[features]
y = data['target']

# Identify and handle missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Sort the data by match_date to ensure correct ordering
data = data.sort_values(by='match_date')

# Define the percentage for training data
train_size = 0.8  # 80% for training, 20% for testing

# Determine the cutoff index for the training set
cutoff_index = int(len(data) * train_size)

# Split the data into training and testing sets based on the cutoff index
train_data = data.iloc[:cutoff_index]
test_data = data.iloc[cutoff_index:]

X_train = train_data[features]
y_train = train_data['target']
X_test = test_data[features]
y_test = test_data['target']

# Impute missing values in train and test sets
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Define the model and hyperparameter grid
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
param_grid = {
    'n_estimators': [200, 400],
    'learning_rate': [0.01, 0.3, 0.5],
    'max_depth': [11, 13, 15],
    'colsample_bytree': [0.2, 0.8]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

# Train the model with the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Save the trained model
joblib.dump(best_model, '/Users/pratikrohila/Desktop/AMEX DATA/xgboost_cricket_model_tuned.pkl')

# Save the label encoder for future use
joblib.dump(label_encoder, '/Users/pratikrohila/Desktop/AMEX DATA/label_encoder.pkl')


Best parameters found:  {'colsample_bytree': 0.2, 'learning_rate': 0.01, 'max_depth': 15, 'n_estimators': 400}
Accuracy: 0.54


['/Users/pratikrohila/Desktop/AMEX DATA/label_encoder.pkl']

In [87]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Load the trained model and label encoder
model = joblib.load('/Users/pratikrohila/Desktop/AMEX DATA/xgboost_cricket_model_tuned.pkl')
label_encoder = joblib.load('/Users/pratikrohila/Desktop/AMEX DATA/label_encoder.pkl')

# Define a function to prepare new match data
def prepare_new_match_data(new_data, label_encoder, features):
    # Encode categorical features
    categorical_columns = ['toss_winner_team_name', 'toss_decision_by_toss_winning_team']
    for column in categorical_columns:
        new_data[column] = new_data[column].astype(str).map(lambda s: s if s in label_encoder.classes_ else '<unknown>')
        new_data[column] = new_data[column].map(lambda s: label_encoder.transform([s])[0] if s in label_encoder.classes_ else -1)
    
    # Identify and handle missing values
    imputer = SimpleImputer(strategy='mean')
    new_data[features] = imputer.fit_transform(new_data[features])
    
    return new_data[features]

# Load the new match data from the provided CSV file
new_file_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Test Data Sample/updated_data.csv'
new_match_data = pd.read_csv(new_file_path)

# Define the feature columns expected by the model
# Replace 'some_other_feature1', 'some_other_feature2' with actual feature names used in training
features = [
    'boundaries_hit_team1_', 'boundaries_hit_team2_',
    'rolling_avg_bowl_strRate_team1_', 'rolling_avg_bowl_strRate_team2_', 
    'rolling_avg_team1_', 'rolling_avg_team2_', 
    'rolling_strike_rate_team1_', 'rolling_strike_rate_team2_', 
    'toss_winner_team_name', 'toss_decision_by_toss_winning_team',
    'wickets_taken_last5_team1_', 'wickets_taken_last5_team2_',
    'team1_id', 'team2_id'
]

# Check if all features are in the new match data
missing_features = [feature for feature in features if feature not in new_match_data.columns]
if missing_features:
    print(f"Missing features in new match data: {missing_features}")
else:
    print("All features are present in the new match data")

# Add dataset_type with default value
new_match_data['dataset_type'] = 'r1'  # or 'train' based on your context

# Ensure we select only the features expected by the model
new_match_data = new_match_data[features + ['match_id', 'dataset_type']]

# Print the shape of the new data to debug
print(f"Shape of new_match_data: {new_match_data[features].shape}")
print(f"Expected number of features: {model.get_booster().num_features()}")

# Prepare the new match data
X_new = prepare_new_match_data(new_match_data, label_encoder, features)

# Make predictions and get probabilities
predictions = model.predict(X_new)
prediction_probs = model.predict_proba(X_new)[:, 1]  # Probability of class 1

# Map predictions back to team1_id or team2_id
new_match_data['win_pred_team_id'] = new_match_data.apply(
    lambda row: row['team1_id'] if predictions[row.name] == 1 else row['team2_id'], axis=1)

# Add win_pred_score (probability)
new_match_data['win_pred_score'] = prediction_probs

# Add fixed variables
new_match_data['train_algorithm'] = 'xgboost'
new_match_data['Ensemble?'] = 'no'
new_match_data['train_hps_trees'] = 100
new_match_data['train_hps_depth'] = 8
new_match_data['train_hps_lr'] = 0.1

# Add top 10 features as dynamic variables
top_10_features = [
    'rolling_avg_bowl_strRate_team1_', 'rolling_avg_bowl_strRate_team2_', 
    'rolling_avg_team1_', 'rolling_avg_team2_', 
    'rolling_strike_rate_team1_', 'rolling_strike_rate_team2_', 
    'toss_winner_team_name', 'toss_decision_by_toss_winning_team', 
    'wickets_taken_last5_team2_', 'wickets_taken_last5_team1_'
]

for i, feature in enumerate(top_10_features, 1):
    new_match_data[f'indep_feat_id{i}'] = new_match_data[feature]

# Create the submission dataframe with required columns
submission = new_match_data[['match_id', 'dataset_type', 'win_pred_team_id', 'win_pred_score', 
                             'train_algorithm', 'Ensemble?', 'train_hps_trees', 'train_hps_depth', 
                             'train_hps_lr'] + [f'indep_feat_id{i}' for i in range(1, 11)]]

# Save the submission dataframe to a new CSV file
output_path = '/Users/pratikrohila/Desktop/AMEX Round 2/Test Data Sample/predicted_winners_New3.csv'
submission.to_csv(output_path, index=False)

# Display the submission dataframe
print(submission.head())


All features are present in the new match data
Shape of new_match_data: (207, 14)
Expected number of features: 14
   match_id dataset_type  win_pred_team_id  win_pred_score train_algorithm  \
0   9272619           r1           33928.0        0.440741         xgboost   
1   9086958           r1              69.0        0.337049         xgboost   
2   9433654           r1            7573.0        0.663494         xgboost   
3   9097248           r1           22763.0        0.379820         xgboost   
4   9097234           r1           23750.0        0.392633         xgboost   

  Ensemble?  train_hps_trees  train_hps_depth  train_hps_lr  indep_feat_id1  \
0        no              100                8           0.1       15.972973   
1        no              100                8           0.1       23.083333   
2        no              100                8           0.1       15.157895   
3        no              100                8           0.1       15.833333   
4        no           